In [9]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import metrics, utils
import pandas as pd
import seaborn as sns
import cv2
from PIL import Image
import os

In [11]:
train_data = pd.read_csv("Train.csv")
test_data = pd.read_csv("Train.csv")

train_data.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png
